# Import

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os, sys
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from get_dataset import get_training_and_validation


# Load Data

In [ ]:
batch_size = 64
epochs = 15
IMG_HEIGHT = 100
IMG_WIDTH = 100

# make sure to write own classifier-function in next cell
# if num_classes is greater than 2
num_classes=2



In [ ]:
# this filter allows all combination with exactly 1 human on it
# artificial masks (photoshop, cg), are not distinguished

# this implicite sets the ['class'] as follows:
#   with_mask >= 1 -> 'mask'
#   with_mask == 0 -> 'no_mask'
#  classifier-attribute can be set to own function for matching data-rows into classes
frame = get_training_and_validation(people_per_img = 1, with_mask=None, no_mask=None, unknown=0)
print('Filter returned {:d} images'.format(len(frame)))

# Data preparation

In [ ]:
def get_data_generator(image_generator, subset=None):
    return image_generator.flow_from_dataframe(frame, directory='./', x_col='Path', y_col='class',
                                                 batch_size = batch_size, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='categorical', subset=subset)

In [ ]:
# generator for all data, reserve 15% of data for training
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15) # Generator for our training data

In [ ]:
train_data_gen = get_data_generator(image_generator, subset='training')

validation_data_gen = get_data_generator(image_generator, subset='validation')


In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

# Create the model

In [ ]:
model = Sequential([
    Conv2D(32, 3, padding='same', strides=2,activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', strides=2,activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', strides=2,activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['acc'])

In [ ]:
model.summary()

# Train the model

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = 1989 // batch_size,
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps = 351 //batch_size
)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Data augmentation

## Flip and Shift

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,width_shift_range=.15,
                    height_shift_range=.15,)

train_data_gen = get_data_generator(image_gen)

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
# Re-use the same custom plotting function defined and used
# above to visualize the training images
plotImages(augmented_images)

## Rotate

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=15)

train_data_gen = get_data_generator(image_gen)

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

## Zoom

In [ ]:
# zoom_range from 0 - 1 where 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [ ]:
train_data_gen = get_data_generator(image_gen)

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmented_images)

## Together

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=15,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.2,
                    validation_split=0.15
                    )

In [ ]:
train_data_gen = get_data_generator(image_gen_train, subset='training')
validation_data_gen = get_data_generator(image_gen_train, subset='validation')


In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

## Validation

In [ ]:
# validation data is generated together with training data
#image_gen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
#val_data_gen = get_data_generator(image_gen_val)

#val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
#                                                 directory=validation_dir,
#                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
#                                                 class_mode='categorical')

# Dropout

In [ ]:
model_new = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.1),
    Conv2D(64, 3, padding='same', strides=2, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', strides=2, activation='relu'),
    MaxPooling2D(),
    Dropout(0.1),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])


In [ ]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

# Train2

In [ ]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=len(train_data_gen),
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps=len(validation_data_gen)
)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Test

In [ ]:
import tensorflow_datasets as tfds

IMG_SIZE = 100 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label


In [ ]:
#os.listdir("D:Test/Test")


In [ ]:
PATH="D:"
test_dir = 'Test'
test_len = len(os.listdir(test_dir))

# TODO: use the new flow_from_dataframe for testing aswell

test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data


test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

    

In [ ]:
#predict=(model.predict(test_data_gen)> 0.5).astype("int32")
predict=model.predict(test_data_gen)
test_dir

In [ ]:
predict

In [ ]:
import glob
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

filelist = glob.glob(os.path.join(test_dir, "Test/*.png"))
#print(filelist)
for idx,f in enumerate(filelist[10:30]):
    img=mpimg.imread(f)
    imgplot = plt.imshow(img)
    plt.title(predict[idx])
    plt.show()

In [ ]:
import glob
from PIL import Image
test_dir ='Test2'

images=glob.glob(os.path.join(test_dir, "Test/*.*"))

print(len(images))
for image in images: 
    pimg=mpimg.imread(image)
    imgplot = plt.imshow(pimg)
    
    
    img=tf.keras.preprocessing.image.load_img(image,target_size=(100,100))
    imgtf=tf.keras.preprocessing.image.img_to_array(img)
    imgtf = np.expand_dims(imgtf, axis=0)
    predict=model.predict(imgtf)
    

    plt.title(predict)
    plt.show()
    
    
    

